# init spark session

In [2]:
from IPython.core.display import HTML
display(HTML("""<style>pre { white-space: pre !important; }.container { width:100% !important; }</style>"""))

In [3]:
from pyspark.sql import SparkSession
from sqlalchemy import create_engine, text
engine = create_engine('trino://trino@host.docker.internal:9090/iceberg')

spark = (
    SparkSession.builder
    .master("yarn")
    .appName("bronze-to-silver")
    .config("spark.driver.memory", "1g")
    .config("hive.metastore.uris", "thrift://hive-metastore:9083")
    .config("spark.sql.warehouse.dir", "hdfs://yarn-master:9000/user/hive/warehouse/default")
    .config("spark.jars", """
        hdfs://yarn-master:9000/user/hive/spark_jars/iceberg-hive-runtime-1.4.3.jar,
        hdfs://yarn-master:9000/user/hive/spark_jars/iceberg-spark-runtime-3.4_2.12-1.4.3.jar
    """)
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
    .config("spark.executor.cores", 3)
    .config("spark.executor.memory", "4g")
    .config("spark.executor.instances", 1)
    .enableHiveSupport()
    .getOrCreate()
)

warehouse_path = "/user/hive/warehouse"
data_source = "product1.public"
biz_key_map = {
    "customer": ["customer_id"],
    "service": ["service_id"],
    "period": ["period_id"],
    "trans": ["trans_id"],
    "review": ["review_id"]
}

/opt/spark/bin/load-spark-env.sh: line 68: ps: command not found
25/09/17 12:15:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/17 12:15:09 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
spark.sql(f"""
    create database if not exists bronze
    location '{warehouse_path}/bronze'
    """)
spark.sql(f"""
    create database if not exists silver
    location '{warehouse_path}/silver'
    """)
spark.sql(f"""
    create database if not exists gold
    location '{warehouse_path}/gold'
    """)

DataFrame[]

In [4]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|   bronze|
|  default|
|     gold|
|   silver|
+---------+



# create new bronze tables

In [5]:
import re

print(f"{'#'*2} - Create new bronze tables")
pattern = re.compile(rf"{data_source}.*")
hadoop = spark.sparkContext._jvm.org.apache.hadoop
fs = hadoop.fs.FileSystem
conf = hadoop.conf.Configuration() 
path = hadoop.fs.Path(f"{warehouse_path}/bronze/topics")
all_paths = [str(f.getPath()).split("/")[-1] for f in fs.get(conf).listStatus(path)]
all_tables = [path for path in all_paths if pattern.match(path)]
print(f"{'#'*4} - all_tables={all_tables}")
for table in all_tables:
    hdfs_path = f"{warehouse_path}/bronze/topics/{table}"
    table_name = table.replace(".", "_")
    df = (
        spark.read.format('parquet')
        .options(header=True, inferSchema=True)
        .load(hdfs_path)
    )
    if spark.sql(f"show tables from bronze like '{table_name}'").isEmpty():
        print(f"{'#'*4} - Create table bronze.{table_name}")
        spark.sql(f"""
            create external table if not exists bronze.{table_name}
            ({', '.join([col + ' ' + dtype for col, dtype in df.dtypes])})
            partitioned by (__ds)
            stored as parquet
            location "{hdfs_path}"
        """)
        spark.sql(f'repair table bronze.{table_name}')

## all_tables=['product1.public.customer']


## create table bronze.product1_public_customer


25/09/17 06:56:53 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [7]:
spark.sql("select * from bronze.product1_public_customer").show(10, False)

[Stage 1:>                                                          (0 + 1) / 1]

+-----------+----------+---------+----------+-------+------------+--------+---------------+-------------------+---------+----+--------+-----------------------+----------+
|customer_id|first_name|last_name|birth_date|address|phone_number|email   |job_title      |updated_datetime   |__deleted|__op|__lsn   |__src_ts_ms            |__ds      |
+-----------+----------+---------+----------+-------+------------+--------+---------------+-------------------+---------+----+--------+-----------------------+----------+
|1          |Vu        |Tran     |2025-09-17|string |1           |vu.tran2|Data Engineer 1|2025-09-17 13:44:15|false    |c   |26920528|2025-09-17 06:44:15.948|2025-09-17|
|1          |Vu        |Tran     |2025-09-17|string |2           |vu.tran2|Data Engineer 1|2025-09-17 13:45:33|false    |u   |26920912|2025-09-17 06:45:33.658|2025-09-17|
|1          |Vu        |Tran     |2025-09-17|string |3           |vu.tran2|Data Engineer 1|2025-09-17 13:45:38|false    |u   |26921096|2025-09-17

# create new silver tables

In [15]:
print(f"{'#'*2} - Create new silver tables")
table_name_pattern = data_source.replace(".", "_")
bronze_table_list = (
    spark.sql(f"show tables from bronze like '{table_name_pattern}_*'")
        .select("tableName")
        .rdd.flatMap(lambda x: x).collect()
)
silver_table_list = (
    spark.sql(f"show tables from silver like '{table_name_pattern}_*'")
        .select("tableName")
        .rdd.flatMap(lambda x: x).collect()
)
new_bronze_tables = list(set(bronze_table_list) - set(silver_table_list))
print(f"{'#'*4} - new_bronze_tables={new_bronze_tables}")

## - Create new silver tables
#### - new_bronze_tables=['product1_public_customer']


In [18]:
for new_bronze_table in new_bronze_tables:
    table_list = {"derived": f"{new_bronze_table}_der", "snapshot": f"{new_bronze_table}_snp"}
    df = spark.sql(f"select * from bronze.{new_bronze_table}")
    for table_type, table_name in table_list.items():
        print(f"{'#'*4} - Create table silver.{table_name}")
        spark.sql(f"""
            create external table if not exists silver.{table_name}
            ({', '.join([col + ' ' + dtype for col, dtype in df.dtypes])})
            using iceberg
            location '{warehouse_path}/silver/{table_name}'
            tblproperties(
                'objcapabilities'='extread,extwrite',
                'engine.hive.enabled'='true',
                'write.delete.mode'='copy-on-write',
                'write.update.mode'='copy-on-write',
                'write.merge.mode'='copy-on-write',
                'external.table.purge'='true',
                'iceberg.file_format'='parquet',
                'format-version'='2',
                'read.parquet.vectorization.batch-size'='10000',
                'read.parquet.vectorization.enabled'='false'
            )
        """)
    create_view_sql = f"""
        create view silver.{new_bronze_table}_spark as
        with ranking as (
            select *,
                row_number() over(
                    partition by {', '.join(biz_key_map[new_bronze_table.replace(f'{table_name_pattern}_', '')])}
                    order by __src_ts_ms desc, __lsn desc
                ) as rn
            from silver.{table_list['derived']}
        )
        select *
        from ranking
        where rn = 1 and __op <> 'd'
    """
    print(f"{'#'*4} - Create view silver.{table_name}_spark")
    spark.sql(create_view_sql)
    print(f"{'#'*4} - Create view silver.{table_name}")
    with engine.connect() as connection:
        connection.execute(text(create_view_sql.replace("_spark", "")))

#### - Create table silver.product1_public_customer_der
#### - Create table silver.product1_public_customer_snp
#### - Create view silver.product1_public_customer_snp_{engine_name} for spark and trino


In [5]:
# spark.sql("drop table silver.product1_public_customer_der")
# spark.sql("drop table silver.product1_public_customer_snp")
# spark.sql("drop view silver.product1_public_customer_spark")

DataFrame[]

# read and write stream

In [4]:
print(f"{'#'*2} - Run streaming")
for bronze_table in bronze_table_list:
    stream_reader = (
        spark.readStream
        .schema(spark.sql(f"select * from bronze.{bronze_table}").schema)
        .parquet(f"{warehouse_path}/bronze/topics/{bronze_table.replace('_', '.')}")
    )
    # stream_writer = (
    #     stream_reader.writeStream
    #     .outputMode("append").format("console")
    #     .start()
    # )
    stream_query = (
        stream_reader.writeStream
        .outputMode("append").format("iceberg")
        .option("checkpointLocation", f"{warehouse_path}/bronze/spark_checkpoints/{bronze_table}")
        .trigger(processingTime="10 seconds")
        .toTable(f"silver.{bronze_table}_der")
    )
spark.streams.awaitAnyTermination()

## - Run streaming


25/09/17 09:54:05 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.10/socket.py", line 717, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [5]:
spark.sql(f"select * from silver.{bronze_table}_der").show(10, False)

+-----------+----------+---------+----------+-------+------------+--------+---------------+-------------------+---------+----+--------+-----------------------+----------+
|customer_id|first_name|last_name|birth_date|address|phone_number|email   |job_title      |updated_datetime   |__deleted|__op|__lsn   |__src_ts_ms            |__ds      |
+-----------+----------+---------+----------+-------+------------+--------+---------------+-------------------+---------+----+--------+-----------------------+----------+
|1          |Vu        |Tran     |2025-09-17|string |1           |vu.tran2|Data Engineer 1|2025-09-17 13:44:15|false    |c   |26920528|2025-09-17 06:44:15.948|2025-09-17|
|1          |Vu        |Tran     |2025-09-17|string |2           |vu.tran2|Data Engineer 1|2025-09-17 13:45:33|false    |u   |26920912|2025-09-17 06:45:33.658|2025-09-17|
|1          |Vu        |Tran     |2025-09-17|string |3           |vu.tran2|Data Engineer 1|2025-09-17 13:45:38|false    |u   |26921096|2025-09-17

In [6]:
spark.sql(f"select * from silver.{bronze_table}").show(10, False)

+-----------+----------+---------+----------+-------+------------+--------+---------------+-------------------+---------+----+--------+-----------------------+----------+---+
|customer_id|first_name|last_name|birth_date|address|phone_number|email   |job_title      |updated_datetime   |__deleted|__op|__lsn   |__src_ts_ms            |__ds      |rn |
+-----------+----------+---------+----------+-------+------------+--------+---------------+-------------------+---------+----+--------+-----------------------+----------+---+
|1          |Vu        |Tran     |2025-09-17|string |3           |vu.tran2|Data Engineer 1|2025-09-17 13:45:38|false    |u   |26921096|2025-09-17 06:45:38.231|2025-09-17|1  |
+-----------+----------+---------+----------+-------+------------+--------+---------------+-------------------+---------+----+--------+-----------------------+----------+---+

